## Data preparation

We will use images downloaded from Sentinel-2 using the download_data.py script. 

In [18]:
import sys
sys.path.append('../util')
import myaugmentor

In [19]:
p = myaugmentor.MyPipeline('../data/tulips/bloom/16/img/')

Initialised with 5621 image(s) found.
Output directory set to ../data/tulips/bloom/16/img/output.

In [20]:
p.ground_truth('../data/tulips/bloom/16/mask')

5621 ground truth image(s) found.


In [21]:
p.rotate90(probability=0.5)
p.rotate270(probability=0.5)
p.flip_left_right(probability=0.8)
p.flip_top_bottom(probability=0.3)

In [22]:
g = p.image_generator()

In [23]:
next(g)

(<PIL.Image.Image image mode=RGB size=256x256 at 0x7FC7C46A1AC8>, 0)